# Homework 2 - Credit Analytics
Submission by Dennis Goldenberg

In [3]:
import numpy as np
import pandas as pd

## $\underline{\text{Question 1}}$

### a. Expected Value of Loss Leg
I use the formula for expectation and the formulas given to calculate:
$$
\begin{align*}
\mathbb{E}\left[\text{PV}(\text{loss})\right] &= \int_0^{\Delta t}f(t)*\text{PV}\left(\text{loss payment}|\text{default at t}\right)dt\\
&= \int_0^{\Delta t}he^{-ht}*(1 - R)e^{-rt}dt\\
&= h(1 - R)\int_0^{\Delta t}e^{-(r + h)t}dt\\
&= h(1 - R)\left[\frac{e^{-(r + h)t}}{-(r + h)}\right]_0^{\Delta t}\\
&= h(1 - R)\left(\frac{1}{r + h} - \frac{e^{-(r + h)\Delta t}}{r + h}\right)\\
&= \frac{h}{r + h}(1 - R)\left(1 - e^{-(r + h)\Delta t}\right)
\end{align*}
$$
Note that this quantity can be multiplied by the notional amount of the underlying bond; here, the implicit assumption is that the notional amount is 1 dollar.

### b. Expected Value of Refund

### c. Fair Value of P for One-period Swap

For this question: Read defaultswap.pdf

## $\underline{\text{Question 2}}$

For this question: read creditportfoliocalculation.pdf

## $\underline{\text{Question 3}}$

For this question: read understandingcopulas.pdf

## $\underline{\text{Question 4}}$

For this question: read AnOverViewofCopulaCDOVerion-6.pdf

## $\underline{\text{Question 5}}$

For this question: read creditportfoliocalculation.pdf

## $\underline{\text{Question 6}}$

For this question: read AnOverViewofCopulaCDOVerion-6.pdf